In [6]:
import os
import numpy as np
import nibabel as nib
# from synthmoprh.voxelmorph.torch.networks import VxmDense
from nibabel.processing import conform
import torch
from scipy.ndimage import center_of_mass

import SimpleITK as sitk
from pathlib import Path
from tqdm import tqdm
from scipy import ndimage
import pandas as pd

import sys
sys.path.append('/home/vzalevskyi/uni/MAIA_Semester_3/mira/final_project/maia-mira/synthmorph')
sys.path.append('/home/vzalevskyi/uni/MAIA_Semester_3/mira/final_project/maia-mira')
from utils import parse_points_reg, compute_TRE

from preprocess_data import register_image
import torch 
import tensorflow as tf
import nibabel as nib
import numpy as np

from utils import print_img_info

from voxelmorph.tf.utils import point_spatial_transformer

# Load preprocessed data (scaled between 0 and 1 and with the moving data in the space of the fixed one)
# fixed = nib.load("processed_data/copd_scans/scans/case_001_insp.nii.gz")

# moving = nib.load("processed_data/copd_scans/scans/case_001_exp.nii.gz")

In [2]:
# fixed_mask = nib.load("processed_data/copd_scans/lungMasks/case_001_insp.nii.gz").get_fdata()

# moving_mask = nib.load("processed_data/copd_scans/lungMasks/case_001_exp.nii.gz").get_fdata()

In [15]:
cx, cy, cz = center_of_mass(fixed_mask)

In [17]:
idxs = [(x.min(), x.max()) for x in np.where(fixed_mask !=0)]
idxs

[(57, 450), (106, 441), (0, 117)]

In [3]:
print(moving.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 512 512 121   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float64
bitpix          : 64
slice_start     : 0
pixdim          : [-1.    0.63  0.63  2.5   0.    0.    0.    0.  ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 2
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : scanner
sform_code      : unknown
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 1.0
qoffset_x       : -0.0
qoffset_y       : -0.0
qoffset_z       : 0.0
srow_x          : [0. 0. 0. 

In [18]:

output_shape = (256, 256, 60)
output_spacing = (1, 1, 1)#(1.2, 1.2, 5)
moving_cropped = nib.processing.conform(moving, out_shape=output_shape, voxel_size=output_spacing)
fixed_cropped = nib.processing.conform(fixed, out_shape=output_shape, voxel_size=output_spacing)

In [19]:
nib.save(moving_cropped, 'moving_cropped.nii.gz')
nib.save(fixed_cropped, 'fixed_cropped.nii.gz')

In [20]:
moving.shape

(512, 512, 121)

In [37]:
fixed_image = sitk.ReadImage("/home/vzalevskyi/uni/MAIA_Semester_3/mira/final_project/maia-mira/data/copd_scans/scans/case_004_insp.nii.gz")
moving_image = sitk.ReadImage('/home/vzalevskyi/uni/MAIA_Semester_3/mira/final_project/maia-mira/data/copd_scans/scans/case_004_exp.nii.gz')


In [38]:
print_img_info(fixed_image)

Train image:
origin: (0.0, 0.0, 0.0)
size: (512, 512, 126)
spacing: (0.5899999737739563, 0.5899999737739563, 2.5)
direction: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0)
pixel type: 16-bit signed integer
number of pixel components: 1


In [39]:
def resample_volume(volume, interpolator = sitk.sitkLinear, new_spacing = [0.63, 0.63, 3]):
    original_spacing = volume.GetSpacing()
    original_size = volume.GetSize()
    new_size = [int(round(osz*ospc/nspc)) for osz,ospc,nspc in zip(original_size, original_spacing, new_spacing)]
    return sitk.Resample(volume, new_size, sitk.Transform(), interpolator,
                         volume.GetOrigin(), new_spacing, volume.GetDirection(), 0,
                         volume.GetPixelID())

In [49]:
moving_iso = resample_volume(moving_image, new_spacing = [0.59, 0.59, 3.29])
print_img_info(moving_image)
print()
print_img_info(moving_iso)

sitk.WriteImage(moving_iso, 'moving_iso.nii.gz')

Train image:
origin: (0.0, 0.0, 0.0)
size: (512, 512, 126)
spacing: (0.5899999737739563, 0.5899999737739563, 2.5)
direction: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0)
pixel type: 16-bit signed integer
number of pixel components: 1

Train image:
origin: (0.0, 0.0, 0.0)
size: (512, 512, 96)
spacing: (0.59, 0.59, 3.29)
direction: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0)
pixel type: 16-bit signed integer
number of pixel components: 1


In [ ]:
oldInd=[120, 150, 90]
p=oldImage.TransformIndexToPhysicalPoint(oldInd)
newInd=newImage.TransformPhysicalPointToIndex(p)
print(newInd)

In [38]:
moving_image.TransformPhysicalPointToIndex(moving_image.TransformIndexToPhysicalPoint((200, 100, 100)))

(200, 100, 100)

TransformContinuousIndexToPhysicalPoint, TransformIndexToPhysicalPoint, TransformPhysicalPointToContinuousIndex, and TransformPhysicalPointToIndex.

In [ ]:
TransformContinuousIndexToPhysicalPoint, TransformIndexToPhysicalPoint, TransformPhysicalPointToContinuousIndex, and TransformPhysicalPointToIndex.